# Plan to medal - Herbarium 2021 (FGVC8)

Objectives:
- Establish robust GPU-enabled pipeline quickly.
- Validate data loading from provided metadata and image structure.
- Build a fast, strong baseline (image classifier with transfer learning, e.g., timm/torchvision).
- Lock CV protocol mirroring test (stratified KFold on species).
- Iterate: augmentations, resolution, label smoothing, class-balanced loss, mixup/cutmix. Cache checkpoints.
- Ensembling if time allows (multi-seed, different backbones).

Milestones:
1) Environment + data sanity checks.
2) EDA: label distribution, image counts, sources, leakage checks.
3) Baseline model: pretrained backbone (e.g., convnext_tiny / efficientnet_b3).
4) CV + OOF + test inference, submission.csv.
5) Error analysis and quick improvements (augmentations, loss, re-balancing).

Notes:
- Metric: macro F1, long-tail: prioritize class-balanced sampling/loss.
- Always log progress and elapsed time per fold.
- Save OOF and test logits for future blends.

Next: Run environment & data checks.

In [21]:
import os, json, random, time, subprocess, sys
from pathlib import Path
import pandas as pd

t0 = time.time()
print("=== GPU check: nvidia-smi ===", flush=True)
try:
    print(subprocess.run(['bash','-lc','nvidia-smi || true'], capture_output=True, text=True).stdout)
except Exception as e:
    print("nvidia-smi failed:", e)

ROOT = Path('.')
TRAIN_DIR = ROOT / 'train'
TEST_DIR = ROOT / 'test'
TRAIN_META = TRAIN_DIR / 'metadata.json'
TEST_META = TEST_DIR / 'metadata.json'
SAMPLE_SUB = ROOT / 'sample_submission.csv'

print("=== Files existence ===", flush=True)
for p in [TRAIN_DIR, TEST_DIR, TRAIN_DIR/'images', TEST_DIR/'images', TRAIN_META, TEST_META, SAMPLE_SUB]:
    print(f"{p}: {'OK' if p.exists() else 'MISSING'}")

def load_coco_like(pth):
    with open(pth, 'r') as f:
        data = json.load(f)
    # Expected keys: annotations, images, categories
    anns = pd.DataFrame(data.get('annotations', []))
    imgs = pd.DataFrame(data.get('images', []))
    cats = pd.DataFrame(data.get('categories', []))
    return anns, imgs, cats

print("\n=== Load train/test COCO-like metadata ===", flush=True)
tr_anns, tr_imgs, tr_cats = load_coco_like(TRAIN_META)
te_anns, te_imgs, te_cats = load_coco_like(TEST_META)  # test usually has images and no annotations
print("train anns/imgs/cats shapes:", tr_anns.shape, tr_imgs.shape, tr_cats.shape)
print("test anns/imgs/cats shapes:", te_anns.shape, te_imgs.shape, te_cats.shape)
print("train anns head:\n", tr_anns.head(3))
print("train imgs head:\n", tr_imgs.head(3))
print("cats head:\n", tr_cats.head(3))

# Identify columns
species_col = None
for c in ['category_id','species_id','label','target']:
    if c in tr_anns.columns:
        species_col = c; break
image_id_col = 'image_id' if 'image_id' in tr_anns.columns else None
specimen_col = None
for c in ['specimen_id','specimen','record_id']:
    if c in tr_imgs.columns:
        specimen_col = c; break
file_col = None
for c in ['file_name','file','path']:
    if c in tr_imgs.columns:
        file_col = c; break

print(f"Detected -> species: {species_col}, specimen: {specimen_col}, file: {file_col}, image_id_col: {image_id_col}")
assert species_col is not None and image_id_col is not None, "Train annotations must have category_id/species and image_id"
assert file_col is not None and 'id' in tr_imgs.columns, "Train images must have id and file_name"

# Merge anns with imgs to get file paths and specimen
train_df = tr_anns.merge(tr_imgs, left_on=image_id_col, right_on='id', how='inner', suffixes=('_ann','_img'))
# Build correct absolute file paths (file_name already includes 'images/...')
train_df['file_path'] = train_df[file_col].apply(lambda x: TRAIN_DIR / x)
te_imgs['file_path'] = te_imgs[file_col].apply(lambda x: TEST_DIR / x)
print("train_df merged shape:", train_df.shape)

print("\n=== Basic stats ===", flush=True)
n_species = train_df[species_col].nunique()
n_imgs = len(train_df)
n_specimens = train_df[specimen_col].nunique() if specimen_col and specimen_col in train_df.columns else None
print("species unique:", n_species, "images:", n_imgs, "specimens unique:", n_specimens)
cnts = train_df[species_col].value_counts()
print("per-class count head:", cnts.head().to_dict())
print("per-class count tail:", cnts.tail().to_dict())

print("\n=== Path existence checks (sample) ===", flush=True)
def build_path(split, rel):
    base = TRAIN_DIR if split=='train' else TEST_DIR
    return base / rel  # file_name already contains 'images/...'
sample_train = train_df.sample(min(20, len(train_df)), random_state=42)
missing_train = 0
for r in sample_train[file_col].tolist():
    p = build_path('train', r)
    if not p.exists():
        missing_train += 1
print(f"Train sample missing paths: {missing_train}/{len(sample_train)}")

print("\n=== Test metadata overview ===", flush=True)
assert 'id' in te_imgs.columns and file_col in te_imgs.columns, "Test images must have id and file_name"
print(te_imgs[[ 'id', file_col ]].head(3))
sample_test = te_imgs.sample(min(20, len(te_imgs)), random_state=42)
missing_test = 0
for r in sample_test[file_col].tolist():
    p = build_path('test', r)
    if not p.exists():
        missing_test += 1
print(f"Test sample missing paths: {missing_test}/{len(sample_test)}")

print("\n=== Sample submission ===", flush=True)
sdf = pd.read_csv(SAMPLE_SUB)
print("sample_submission head:\n", sdf.head())
print("Elapsed: %.2fs" % (time.time()-t0), flush=True)

=== GPU check: nvidia-smi ===


Mon Sep 29 01:51:45 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.144.06             Driver Version: 550.144.06     CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A10-24Q                 On  |   00000002:00:00.0 Off |                    0 |
| N/A   N/A    P0             N/A /  N/A  |    6722MiB /  24512MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

train: OK
test: OK
train/images: OK
test/images: OK
train/metadata.json: OK
test/metadata.json: OK
sample_submission.csv: OK

=== Load train/test COCO-like metadata ===


train anns/imgs/cats shapes: (1779953, 4) (1779953, 5) (64500, 4)
test anns/imgs/cats shapes: (0, 0) (477806, 5) (0, 0)
train anns head:
    category_id       id  image_id  institution_id
0        60492  1608432   1608432               0
1        60492   796948    796948               0
2        60492   994447    994447               0
train imgs head:
                    file_name  height       id  license  width
0  images/604/92/1608432.jpg    1000  1608432        0    671
1   images/604/92/796948.jpg    1000   796948        0    678
2   images/604/92/994447.jpg    1000   994447        0    671
cats head:
         family  id                             name        order
0  Orchidaceae   0    Aa calceata (Rchb.f.) Schltr.  Asparagales
1  Orchidaceae   1  Aa matthewsii (Rchb.f.) Schltr.  Asparagales
2  Orchidaceae   2      Aa paleacea (Kunth) Rchb.f.  Asparagales
Detected -> species: category_id, specimen: None, file: file_name, image_id_col: image_id


train_df merged shape: (1779953, 10)

=== Basic stats ===


species unique: 64500 images: 1779953 specimens unique: None
per-class count head: {42811: 2647, 25229: 1713, 48372: 1630, 42843: 1324, 22344: 1260}
per-class count tail: {6742: 1, 57300: 1, 44413: 1, 4970: 1, 37044: 1}

=== Path existence checks (sample) ===


Train sample missing paths: 0/20

=== Test metadata overview ===


  id         file_name
0  0  images/000/0.jpg
1  1  images/000/1.jpg
2  2  images/000/2.jpg
Test sample missing paths: 0/20

=== Sample submission ===


sample_submission head:
    Id  Predicted
0   0          0
1   1          0
2   2          0
3   3          0
4   4          0
Elapsed: 14.35s


In [ ]:
import json, time
from collections import Counter
import pandas as pd
from pathlib import Path

t0 = time.time()
TRAIN_META = Path('train/metadata.json')
TEST_META = Path('test/metadata.json')
SAMPLE_SUB = Path('sample_submission.csv')

print('=== Build majority-class baseline submission ===', flush=True)
print('Loading train annotations to compute majority class...', flush=True)
with open(TRAIN_META, 'r') as f:
    data = json.load(f)
anns = data.get('annotations', [])
cnt = Counter(a['category_id'] for a in anns)
mode_cat, mode_cnt = cnt.most_common(1)[0]
n_classes = len(data.get('categories', []))
print(f'Mode category_id: {mode_cat} with count {mode_cnt}; total classes: {n_classes}', flush=True)

print('Loading sample_submission to get test Id order...', flush=True)
sdf = pd.read_csv(SAMPLE_SUB)
print('sample_submission shape:', sdf.shape, 'cols:', sdf.columns.tolist())

sub = pd.DataFrame({'Id': sdf['Id'].values, 'Predicted': mode_cat})
sub.to_csv('submission.csv', index=False)
print('Wrote submission.csv')
print(sub.head())
print('Elapsed: %.2fs' % (time.time()-t0))

In [ ]:
import os, sys, subprocess, shutil, time
from pathlib import Path

t0 = time.time()
print("=== Install CUDA 12.1 torch stack + vision deps ===", flush=True)

def pip(*args):
    print(">", *args, flush=True)
    subprocess.run([sys.executable, "-m", "pip", *args], check=True)

# Uninstall any existing torch stack (idempotent)
for pkg in ("torch","torchvision","torchaudio"):
    subprocess.run([sys.executable, "-m", "pip", "uninstall", "-y", pkg], check=False)

# Clean possible stray site dirs (idempotent)
for d in (
    "/app/.pip-target/torch",
    "/app/.pip-target/torchvision",
    "/app/.pip-target/torchaudio",
    "/app/.pip-target/torch-2.4.1.dist-info",
    "/app/.pip-target/torchvision-0.19.1.dist-info",
    "/app/.pip-target/torchaudio-2.4.1.dist-info",
):
    if os.path.exists(d):
        print("Removing", d); shutil.rmtree(d, ignore_errors=True)

# Install exact cu121 torch stack
pip("install",
    "--index-url", "https://download.pytorch.org/whl/cu121",
    "--extra-index-url", "https://pypi.org/simple",
    "torch==2.4.1", "torchvision==0.19.1", "torchaudio==2.4.1")

# Freeze torch versions for subsequent installs
Path("constraints.txt").write_text("\n".join([
    "torch==2.4.1",
    "torchvision==0.19.1",
    "torchaudio==2.4.1"
]))

# Install vision deps respecting constraints (avoid upgrading torch)
pip("install", "-c", "constraints.txt",
    "timm==1.0.9",
    "albumentations==1.4.14",
    "opencv-python-headless==4.10.0.84",
    "numpy", "pandas", "scikit-learn",
    "accelerate==0.34.2",
    "wandb==0.17.9",
    "einops==0.8.0",
    "--upgrade-strategy", "only-if-needed")

import torch
print("torch:", torch.__version__, "built CUDA:", getattr(torch.version, "cuda", None))
print("CUDA available:", torch.cuda.is_available())
assert str(getattr(torch.version, "cuda", "")).startswith("12.1"), f"Wrong CUDA build: {torch.version.cuda}"
assert torch.cuda.is_available(), "CUDA not available"
print("GPU:", torch.cuda.get_device_name(0))
print("Elapsed install: %.1fs" % (time.time()-t0), flush=True)

In [23]:
import os, time, math, random, json, gc, sys
from pathlib import Path
import numpy as np
import pandas as pd
import cv2
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
from sklearn.metrics import f1_score
from sklearn.model_selection import StratifiedKFold
import timm
from timm.data.mixup import Mixup
from timm.loss import SoftTargetCrossEntropy
from timm.utils.model_ema import ModelEmaV2
from PIL import Image
import torchvision.transforms as T
from torch.optim.lr_scheduler import LinearLR, CosineAnnealingLR, SequentialLR

torch.backends.cudnn.benchmark = True
torch.set_float32_matmul_precision('high')

ROOT = Path('.')
TRAIN_DIR = ROOT / 'train'
TEST_DIR = ROOT / 'test'
SAMPLE_SUB = ROOT / 'sample_submission.csv'

# Use already loaded metadata via fast reload (small cost compared to images I/O)
with open(TRAIN_DIR / 'metadata.json','r') as f: tr_data = json.load(f)
with open(TEST_DIR / 'metadata.json','r') as f: te_data = json.load(f)
tr_anns = pd.DataFrame(tr_data['annotations'])
tr_imgs = pd.DataFrame(tr_data['images'])
tr_cats = pd.DataFrame(tr_data['categories'])
te_imgs = pd.DataFrame(te_data['images'])

# Merge to get file paths
train_df = tr_anns.merge(tr_imgs, left_on='image_id', right_on='id', how='inner')
train_df['file_path'] = train_df['file_name'].apply(lambda x: TRAIN_DIR / x)
te_imgs['file_path'] = te_imgs['file_name'].apply(lambda x: TEST_DIR / x)

# Label encoding
cats = sorted(train_df['category_id'].unique())
cat2lbl = {c:i for i,c in enumerate(cats)}
lbl2cat = {i:c for c,i in cat2lbl.items()}
train_df['label'] = train_df['category_id'].map(cat2lbl)
NUM_CLASSES = len(cats)
print('NUM_CLASSES:', NUM_CLASSES, 'train images:', len(train_df))

# Transforms using torchvision (avoid albumentations dependency issues)
IMG_SIZE = 256
mean = (0.485, 0.456, 0.406)
std = (0.229, 0.224, 0.225)

train_tfms_clean = T.Compose([
    T.RandomResizedCrop(IMG_SIZE, scale=(0.6, 1.0), ratio=(0.75, 1.33)),
    T.RandomHorizontalFlip(p=0.5),
    T.ToTensor(),
    T.Normalize(mean=mean, std=std),
])
train_tfms_aug = T.Compose([
    T.RandomResizedCrop(IMG_SIZE, scale=(0.6, 1.0), ratio=(0.75, 1.33)),
    T.RandomHorizontalFlip(p=0.5),
    T.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.1, hue=0.05),
    T.ToTensor(),
    T.Normalize(mean=mean, std=std),
])
val_tfms = T.Compose([
    T.Resize(IMG_SIZE),
    T.CenterCrop(IMG_SIZE),
    T.ToTensor(),
    T.Normalize(mean=mean, std=std),
])

class HerbDataset(Dataset):
    def __init__(self, df, tfms, is_train=False):
        self.df = df.reset_index(drop=True)
        self.tfms = tfms
        self.is_train = is_train
    def __len__(self):
        return len(self.df)
    def __getitem__(self, i):
        row = self.df.iloc[i]
        fp = row['file_path']
        try:
            img = Image.open(fp).convert('RGB')
        except Exception:
            img = Image.fromarray(np.zeros((IMG_SIZE, IMG_SIZE, 3), dtype=np.uint8))
        img = self.tfms(img)
        label = int(row['label'])
        return img, label

def seed_everything(seed=42):
    random.seed(seed); np.random.seed(seed); torch.manual_seed(seed); torch.cuda.manual_seed_all(seed)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
seed_everything(42)

# Build folds: exclude singleton classes from validation; 2-fold stratified on labels with >=2 samples
label_counts = train_df['label'].value_counts()
valid_labels = label_counts[label_counts >= 2].index
idx_valid = train_df[train_df['label'].isin(valid_labels)].index.values
y_valid = train_df.loc[idx_valid, 'label'].values
singletons_idx = train_df[~train_df['label'].isin(valid_labels)].index.values
print('Singletons (train-only):', len(singletons_idx))
skf = StratifiedKFold(n_splits=2, shuffle=True, random_state=42)
folds = []
for tr_sub, va_sub in skf.split(idx_valid, y_valid):
    tr_idx = idx_valid[tr_sub]
    va_idx = idx_valid[va_sub]
    tr_idx_full = np.concatenate([tr_idx, singletons_idx])
    folds.append((tr_idx_full, va_idx))
print('Prepared folds:', len(folds), '| fold0 train/val sizes:', len(folds[0][0]), len(folds[0][1]))

def build_model():
    model = timm.create_model('convnext_tiny.fb_in22k', pretrained=True, num_classes=NUM_CLASSES, drop_path_rate=0.1)
    model.to(device)
    model = model.to(memory_format=torch.channels_last)
    return model

def evaluate(model, loader, max_batches=None):
    m = model
    m.eval()
    preds, targs = [], []
    with torch.no_grad():
        for b, (imgs, labels) in enumerate(loader):
            imgs = imgs.to(device, non_blocking=True, memory_format=torch.channels_last)
            labels = labels.to(device, non_blocking=True)
            logits = m(imgs)
            pred = logits.argmax(1)
            preds.append(pred.detach().cpu().numpy())
            targs.append(labels.detach().cpu().numpy())
            if max_batches is not None and (b+1) >= max_batches:
                break
    preds = np.concatenate(preds); targs = np.concatenate(targs)
    f1 = f1_score(targs, preds, average='macro')
    return f1

def run_fold(fold, train_idx, val_idx, epochs=9, batch_size=64, accum_steps=4):
    print(f"\n=== Fold {fold} ===", flush=True)
    tr_df = train_df.loc[train_idx]
    va_df = train_df.loc[val_idx]
    # Datasets (start with clean tfms, will toggle to aug later epochs)
    ds_tr = HerbDataset(tr_df, train_tfms_clean, is_train=True)
    ds_va = HerbDataset(va_df, val_tfms, is_train=False)
    # Mini-val: random 1 image per class (cap at 10k) for stable early F1
    mini_va = va_df.groupby('label', group_keys=False).apply(lambda g: g.sample(1, random_state=42))
    if len(mini_va) > 10000:
        mini_va = mini_va.sample(10000, random_state=42)
    ds_mini = HerbDataset(mini_va, val_tfms, is_train=False)
    # Balanced sampling per fold (1/sqrt(freq)), epoch length 300k
    counts = tr_df['label'].value_counts()
    w = tr_df['label'].map(lambda x: 1.0 / (counts[x] ** 0.5)).values
    sampler = WeightedRandomSampler(w, num_samples=300000, replacement=True)
    dl_tr = DataLoader(ds_tr, batch_size=batch_size, sampler=sampler, shuffle=False,
                       num_workers=12, pin_memory=True, persistent_workers=True,
                       prefetch_factor=2, drop_last=True)
    dl_va = DataLoader(ds_va, batch_size=batch_size, shuffle=False, num_workers=8, pin_memory=True, persistent_workers=True)
    dl_mini = DataLoader(ds_mini, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True, persistent_workers=True)

    model = build_model()
    # Differential LRs: higher LR for head, lower for backbone
    head_names = ['head','classifier','fc']
    head_params, backbone_params = [], []
    for n,p in model.named_parameters():
        (head_params if any(h in n for h in head_names) else backbone_params).append(p)
    optimizer = torch.optim.AdamW([
        {'params': backbone_params, 'lr': 5e-4},
        {'params': head_params, 'lr': 5e-3}
    ], weight_decay=0.05)
    # Scheduler with warmup then cosine (step once per epoch)
    warmup = LinearLR(optimizer, start_factor=0.1, total_iters=1)
    cosine = CosineAnnealingLR(optimizer, T_max=max(1, epochs-1), eta_min=1e-6)
    scheduler = SequentialLR(optimizer, [warmup, cosine], milestones=[1])
    # EMA
    ema = ModelEmaV2(model, decay=0.999, device=device)
    # Losses and mixup setup
    criterion_ce = nn.CrossEntropyLoss(label_smoothing=0.1).to(device)
    criterion_soft = SoftTargetCrossEntropy().to(device)
    mixup_fn = Mixup(mixup_alpha=0.3, cutmix_alpha=0.3, prob=0.0, mode='batch', label_smoothing=0.0, num_classes=NUM_CLASSES)
    scaler = torch.amp.GradScaler('cuda', enabled=True)
    best_score = -1.0
    best_f1 = -1.0
    best_mini = -1.0
    best_path = f'ckpt_fold{fold}.pt'
    global_step = 0

    for epoch in range(1, epochs+1):
        t0 = time.time()
        model.train()
        # Toggle training transforms: enable ColorJitter from epoch 3+
        ds_tr.tfms = train_tfms_aug if epoch >= 3 else train_tfms_clean
        # Mixup schedule: 1-3 off (CE+LS), 4..E-1 on (SoftTarget), last off (CE+LS)
        if epoch <= 3:
            mixup_fn.prob = 0.0
            criterion = criterion_ce
        elif epoch < epochs:
            mixup_fn.prob = 0.3
            criterion = criterion_soft
        else:
            mixup_fn.prob = 0.0
            criterion = criterion_ce

        optimizer.zero_grad(set_to_none=True)
        running_loss = 0.0
        steps_this_epoch = 0
        for step, (imgs, labels) in enumerate(dl_tr):
            imgs = imgs.to(device, non_blocking=True, memory_format=torch.channels_last)
            labels = labels.to(device, non_blocking=True)
            with torch.amp.autocast('cuda', enabled=True):
                if mixup_fn.prob > 0.0:
                    imgs_m, targets = mixup_fn(imgs, labels)
                    logits = model(imgs_m)
                    loss = criterion(logits, targets)
                else:
                    logits = model(imgs)
                    loss = criterion(logits, labels)
            loss = loss / accum_steps
            scaler.scale(loss).backward()
            steps_this_epoch += 1
            if (step + 1) % accum_steps == 0:
                scaler.unscale_(optimizer)
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
                scaler.step(optimizer)
                scaler.update()
                ema.update(model)
                optimizer.zero_grad(set_to_none=True)
            running_loss += loss.item() * accum_steps
            global_step += 1
            if step % 50 == 0:
                elapsed = time.time()-t0
                print(f"Fold {fold} Epoch {epoch} Step {step} Loss {running_loss/(step+1):.4f} Elapsed {elapsed:.1f}s", flush=True)
        # Flush remaining grads if last micro-batch didn't hit accum boundary
        remainder = steps_this_epoch % accum_steps
        if remainder != 0:
            scaler.unscale_(optimizer)
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            scaler.step(optimizer)
            scaler.update()
            ema.update(model)
            optimizer.zero_grad(set_to_none=True)

        # Evaluate using EMA weights
        print("Evaluating on mini-val (1/img per class, cap 10k)", flush=True)
        mini_f1 = evaluate(ema.module, dl_mini, max_batches=None)
        # Cap full val; increase cap on final epoch for better selection stability
        max_val_batches = 1500 if epoch == epochs else 500
        print(f"Evaluating on capped validation: {max_val_batches} batches", flush=True)
        f1 = evaluate(ema.module, dl_va, max_batches=max_val_batches)
        sel_score = 0.7 * mini_f1 + 0.3 * f1
        ep_time = time.time()-t0
        print(f"Fold {fold} Epoch {epoch} mini-F1 {mini_f1:.5f} | F1 {f1:.5f} | sel {sel_score:.5f} time {ep_time:.1f}s", flush=True)
        if sel_score > best_score:
            best_score = sel_score
            best_f1 = f1
            best_mini = mini_f1
            torch.save({'model': model.state_dict(), 'ema': ema.state_dict(), 'f1': f1, 'mini_f1': mini_f1, 'sel': sel_score}, best_path)
            print(f"Saved best to {best_path}", flush=True)
        # Step scheduler once per epoch
        scheduler.step()
        gc.collect(); torch.cuda.empty_cache()
    print(f"Fold {fold} best sel {best_score:.5f} (mini {best_mini:.5f} | f1 {best_f1:.5f})", flush=True)
    return best_f1

def build_test_df_in_submission_order():
    sub = pd.read_csv(SAMPLE_SUB)
    # Ensure consistent dtypes for merge
    sub['Id'] = sub['Id'].astype(int)
    te_tmp = te_imgs.copy()
    te_tmp['id'] = te_tmp['id'].astype(int)
    df = sub[['Id']].merge(te_tmp[['id','file_path']], left_on='Id', right_on='id', how='left')
    return df

def infer_fold(fold, batch_size=128):
    ckpt_path = f'ckpt_fold{fold}.pt'
    state = torch.load(ckpt_path, map_location='cpu')
    model = build_model()
    model.load_state_dict(state['model'], strict=True)
    # Rebuild EMA and load
    ema = ModelEmaV2(model, decay=0.999, device=device)
    if 'ema' in state:
        ema.load_state_dict(state['ema'], strict=False)
    m = ema.module if 'ema' in state else model
    m.eval()
    test_df = build_test_df_in_submission_order()
    # Build a test dataset that returns tensors (dummy labels)
    tmp = test_df.copy()
    tmp['label'] = 0
    class TestDataset(Dataset):
        def __init__(self, df, tfms):
            self.df = df.reset_index(drop=True); self.tfms = tfms
        def __len__(self): return len(self.df)
        def __getitem__(self, i):
            fp = self.df.iloc[i]['file_path']
            try:
                img = Image.open(fp).convert('RGB')
            except Exception:
                img = Image.fromarray(np.zeros((IMG_SIZE, IMG_SIZE, 3), dtype=np.uint8))
            img = self.tfms(img)
            return img, 0
    ds_te = TestDataset(tmp, val_tfms)
    dl_te = DataLoader(ds_te, batch_size=batch_size, shuffle=False, num_workers=8, pin_memory=True, persistent_workers=True)
    pred_lbl = []
    with torch.no_grad():
        for imgs, _ in dl_te:
            imgs = imgs.to(device, non_blocking=True, memory_format=torch.channels_last)
            # TTA 2x: center + hflip
            logits1 = m(imgs)
            imgs_flipped = torch.flip(imgs, dims=[3])
            logits2 = m(imgs_flipped)
            logits = (logits1 + logits2) / 2.0
            pred_lbl.append(logits.argmax(1).detach().cpu().numpy())
    pred_lbl = np.concatenate(pred_lbl).astype(np.int32)
    np.save(f'test_predlbl_fold{fold}.npy', pred_lbl)
    return pred_lbl

# Orchestrate: run folds and generate submission
def train_and_predict(epochs=9, batch_size=64, run_folds=(0,1)):
    fold_scores = []
    for fold, (tr, va) in enumerate(folds):
        if fold not in run_folds:
            continue
        print(f"Starting fold {fold} with {len(tr)} train idx and {len(va)} val idx", flush=True)
        f1 = run_fold(fold, tr, va, epochs=epochs, batch_size=batch_size, accum_steps=4)
        fold_scores.append((fold, f1))
    print('Fold scores:', fold_scores, flush=True)
    # Inference and ensembling (majority vote on per-fold label preds)
    fold_preds = []
    for fold, _ in fold_scores:
        preds = infer_fold(fold, batch_size=128)
        fold_preds.append(preds)
    if len(fold_preds) == 1:
        vote = fold_preds[0]
    else:
        stack = np.stack(fold_preds, 0)
        vote = np.apply_along_axis(lambda x: np.bincount(x).argmax(), 0, stack)
    pred_cat = [lbl2cat[int(x)] for x in vote]
    sub = pd.DataFrame({ 'Id': pd.read_csv(SAMPLE_SUB)['Id'].values, 'Predicted': pred_cat })
    sub.to_csv('submission.csv', index=False)
    print('Wrote submission.csv with shape', sub.shape, flush=True)

print('Ready: IMG_SIZE=256; CV=2 folds with singleton-excluded val. CE+LS epochs 1-3; mixup on from epoch 4..E-1; last epoch CE+LS; 300k samples/epoch; EMA=0.999; grad accumulation x4; scheduler step per epoch. Checkpoint by 0.7*mini + 0.3*capped F1; final epoch cap 1500 batches.', flush=True)

NUM_CLASSES: 64500 train images: 1779953
Singletons (train-only): 7745


Prepared folds: 2 | fold0 train/val sizes: 893849 886104
Ready: IMG_SIZE=256; CV=2 folds with singleton-excluded val. CE+LS epochs 1-3; mixup on from epoch 4..E-1; last epoch CE+LS; 300k samples/epoch; EMA=0.999; grad accumulation x4; scheduler step per epoch. Checkpoint by 0.7*mini + 0.3*capped F1; final epoch cap 1500 batches.


In [ ]:
# Quick smoke run on a small subset to validate pipeline end-to-end
import time, numpy as np, pandas as pd
start_time = time.time()
print('Preparing subset for smoke run...', flush=True)

# Subset size (keep small to finish fast); training still outputs 64.5k classes
N = 20000
if len(train_df) > N:
    sub_df = train_df.sample(N, random_state=42).reset_index(drop=True)
    train_df = sub_df
    # Recompute stratification bins on subset
    cls_counts = train_df['category_id'].value_counts()
    freq = train_df['category_id'].map(cls_counts)
    try:
        y_strat = pd.qcut(freq, q=10, duplicates='drop').astype(str)
    except Exception:
        y_strat = pd.cut(freq, bins=10, include_lowest=True).astype(str)
    # Rebuild folds on subset
    skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
    folds = list(skf.split(train_df.index.values, y_strat))
    print('Subset prepared:', len(train_df), 'rows; folds rebuilt:', len(folds), flush=True)
else:
    print('Dataset smaller than subset target; using full set.', flush=True)

print('Starting quick smoke run (1 epoch, fold 0)...', flush=True)
train_and_predict(epochs=1, batch_size=16, run_folds=(0,))
print('Smoke run elapsed: %.1f min' % ((time.time()-start_time)/60.0), flush=True)

In [ ]:
import sys, subprocess, time
t0=time.time()
print('Fixing albumentations/albucore mismatch...', flush=True)
def pip(*args):
    print('>', *args, flush=True)
    subprocess.run([sys.executable, '-m', 'pip', *args], check=True)
subprocess.run([sys.executable, '-m', 'pip', 'uninstall', '-y', 'albucore'], check=False)
pip('install', '-c', 'constraints.txt', 'albumentations==1.3.1', '--upgrade-strategy', 'only-if-needed')
import albumentations as A
print('albumentations version:', A.__version__)
print('Done in %.1fs' % (time.time()-t0))

In [ ]:
import sys, subprocess, shutil, os, time
t0=time.time()
print('Force-reinstall albumentations 1.3.1 and remove albucore remnants...', flush=True)
def run(*args, check=True):
    print('>', *args, flush=True)
    subprocess.run([sys.executable, '-m', 'pip', *args], check=check)

# Uninstall both to avoid mixed files
run('uninstall', '-y', 'albumentations')
run('uninstall', '-y', 'albucore')

# Clean any stray site dirs that might shadow
for d in (
    '/app/.pip-target/albumentations',
    '/app/.pip-target/albumentations-1.4.14.dist-info',
    '/app/.pip-target/albumentations-1.3.1.dist-info',
    '/app/.pip-target/albucore',
    '/app/.pip-target/albucore-0.0.33.dist-info',
):
    if os.path.exists(d):
        print('Removing', d); shutil.rmtree(d, ignore_errors=True)

# Force reinstall desired version (no albucore dependency)
run('install', 'albumentations==1.3.1', '--no-cache-dir', '--upgrade', '--force-reinstall')

import albumentations as A
print('albumentations version:', A.__version__)
print('Done in %.1fs' % (time.time()-t0), flush=True)

In [24]:
# Train-only extension to 9 epochs (avoid heavy test inference here)
import time
t0 = time.time()
print('Training fold 0 @256, bs=64, epochs=9 (no test inference at end)...', flush=True)
tr_idx, va_idx = folds[0]
best_f1 = run_fold(0, tr_idx, va_idx, epochs=9, batch_size=64, accum_steps=4)
print('Fold0 training complete. Best val F1:', best_f1, flush=True)
print('Elapsed: %.1f min' % ((time.time()-t0)/60.0), flush=True)

Training fold 0 @256, bs=64, epochs=9 (no test inference at end)...



=== Fold 0 ===


/tmp/ipykernel_222/4113945092.py:145: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  mini_va = va_df.groupby('label', group_keys=False).apply(lambda g: g.sample(1, random_state=42))


Fold 0 Epoch 1 Step 0 Loss 11.0847 Elapsed 1.7s


Fold 0 Epoch 1 Step 50 Loss 11.0946 Elapsed 9.8s


...

Fold 0 Epoch 9 Step 3100 Loss 2.7272 Elapsed 521.6s


Fold 0 Epoch 9 Step 3150 Loss 2.7259 Elapsed 530.0s


Fold 0 Epoch 9 Step 3200 Loss 2.7243 Elapsed 538.4s


Fold 0 Epoch 9 Step 3250 Loss 2.7235 Elapsed 546.8s


Fold 0 Epoch 9 Step 3300 Loss 2.7221 Elapsed 555.2s


In [7]:
# Sanity check: try opening a sample of image paths to confirm I/O works
from PIL import Image
import numpy as np
import random, time
from pathlib import Path

def check_opens(df, n=500, split='train'):
    samp = df.sample(min(n, len(df)), random_state=42).reset_index(drop=True)
    ok = 0; fail = 0; failed_paths = []
    t0 = time.time()
    for i in range(len(samp)):
        fp = samp.iloc[i]['file_path']
        try:
            with Image.open(fp) as im:
                im.verify()  # lightweight check
            ok += 1
        except Exception as e:
            fail += 1
            if len(failed_paths) < 10:
                failed_paths.append((str(fp), str(e)))
    print(f"[{split}] Tried {len(samp)} -> OK {ok}, FAIL {fail}, time {time.time()-t0:.1f}s", flush=True)
    if failed_paths:
        print('Sample failures:')
        for p,e in failed_paths:
            print(' -', p, '|', e)

print('=== Image open sanity checks ===', flush=True)
check_opens(train_df, n=500, split='train')
check_opens(te_imgs, n=200, split='test')

=== Image open sanity checks ===


[train] Tried 500 -> OK 500, FAIL 0, time 0.1s


[test] Tried 200 -> OK 200, FAIL 0, time 0.0s


In [17]:
# Inference from saved best checkpoint for fold 0 and write submission
import pandas as pd
print('Running inference with ckpt_fold0.pt ...', flush=True)
preds = infer_fold(0, batch_size=128)
sub_ids = pd.read_csv(SAMPLE_SUB)['Id'].values
pred_cat = [lbl2cat[int(x)] for x in preds]
sub = pd.DataFrame({'Id': sub_ids, 'Predicted': pred_cat})
sub.to_csv('submission.csv', index=False)
print('submission.csv written with shape', sub.shape, flush=True)

Running inference with ckpt_fold0.pt ...


/tmp/ipykernel_222/3574474352.py:268: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state = torch.load(ckpt_path, map_location='cpu')


KeyboardInterrupt: 

In [19]:
# Retrieval baseline (DINO/ViT features + FAISS flat IP). Prep while classifier trains.
import sys, subprocess, math, time, faiss, torch, numpy as np, pandas as pd
from pathlib import Path
import torchvision.transforms as T
from PIL import Image
import timm
from torch.utils.data import Dataset, DataLoader

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

RET_IMG_SIZE = 256
ret_tfms = T.Compose([
    T.Resize(RET_IMG_SIZE),
    T.CenterCrop(RET_IMG_SIZE),
    T.ToTensor(),
    T.Normalize((0.485,0.456,0.406),(0.229,0.224,0.225)),
])

class ImageTensorDS(Dataset):
    def __init__(self, df, tfms):
        self.df = df.reset_index(drop=True); self.tfms = tfms
    def __len__(self): return len(self.df)
    def __getitem__(self, i):
        fp = self.df.iloc[i]['file_path']
        try:
            img = Image.open(fp).convert('RGB')
        except Exception:
            img = Image.fromarray(np.zeros((RET_IMG_SIZE, RET_IMG_SIZE,3), np.uint8))
        return self.tfms(img), i

def build_feat_model():
    # Prefer fast/compact backbone first to reduce download/compute
    names = [
        'vit_base_patch16_224.dino',
        'convnext_base.fb_in22k',
    ]
    last_err = None
    for name in names:
        try:
            m = timm.create_model(name, pretrained=True, num_classes=0)  # features only
            m.to(device); m.eval(); m = m.to(memory_format=torch.channels_last)
            dummy = torch.zeros(1,3,RET_IMG_SIZE,RET_IMG_SIZE, device=device).to(memory_format=torch.channels_last)
            with torch.no_grad(), torch.amp.autocast('cuda', enabled=torch.cuda.is_available()):
                out = m(dummy)
            d = out.shape[-1] if out.ndim==2 else out.flatten(1).shape[-1]
            return m, d, name
        except Exception as e:
            last_err = e
            continue
    raise RuntimeError(f'No backbone available: {last_err}')

def extract_embeddings(model, loader, d):
    embs = np.zeros((len(loader.dataset), d), dtype=np.float32)
    t0=time.time()
    with torch.no_grad():
        for b, (imgs, idx) in enumerate(loader):
            imgs = imgs.to(device, non_blocking=True, memory_format=torch.channels_last)
            with torch.amp.autocast('cuda', enabled=torch.cuda.is_available()):
                out = model(imgs)
            if out.ndim>2:
                out = out.flatten(1)
            out = torch.nn.functional.normalize(out.float(), dim=1)
            embs[idx.numpy()] = out.detach().cpu().numpy().astype(np.float32)
            if b % 50 == 0:
                print(f'Emb batch {b} / {len(loader)} elapsed {time.time()-t0:.1f}s', flush=True)
    return embs

def build_prototypes(train_df, per_class=4, batch_size=128):
    # Use at most 1 image per class (random) for speed on huge label space
    per_class_eff = 1
    grp = (train_df.groupby('label', group_keys=False)
           .apply(lambda g: g.sample(min(len(g), per_class_eff), random_state=42))
           .reset_index(drop=True))
    print('Prototype sampling rows:', len(grp), flush=True)
    ds = ImageTensorDS(grp, ret_tfms)
    dl = DataLoader(ds, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True, persistent_workers=False)
    model, d, bname = build_feat_model()
    print(f'Feature backbone: {bname}, dim={d}', flush=True)
    embs = extract_embeddings(model, dl, d)
    # One prototype per class (mean of sampled; with 1, it's the vector itself)
    proto_vecs, proto_lbls = [], []
    for lbl, sub in grp.groupby('label'):
        v = embs[sub.index.values]
        proto_vecs.append(v.mean(axis=0))
        proto_lbls.append(int(lbl))
    proto = np.stack(proto_vecs).astype(np.float32)
    proto = proto / np.linalg.norm(proto, axis=1, keepdims=True).clip(1e-9, None)
    return proto, np.array(proto_lbls, dtype=np.int32), d, bname

def build_faiss_index(proto):
    d = proto.shape[1]
    index = faiss.IndexFlatIP(d)
    index.add(proto)
    return index

def run_retrieval_submission(per_class=4, test_batch=128):
    tr = train_df[['file_path','label']].copy()
    te = build_test_df_in_submission_order()
    # Build prototypes (internally uses 1 per class for speed)
    t0 = time.time()
    proto, proto_lbls, d, bname = build_prototypes(tr, per_class=per_class, batch_size=128)
    print(f'Prototypes: {len(proto)} built in {time.time()-t0:.1f}s', flush=True)
    index = build_faiss_index(proto)
    # Test embeddings
    ds_te = ImageTensorDS(te, ret_tfms)
    dl_te = DataLoader(ds_te, batch_size=min(test_batch, 64), shuffle=False, num_workers=4, pin_memory=True, persistent_workers=False)
    model, d2, _ = build_feat_model()
    assert d2 == d, 'Backbone mismatch'
    t1 = time.time()
    preds = []
    with torch.no_grad():
        for b, (imgs, idx) in enumerate(dl_te):
            imgs = imgs.to(device, non_blocking=True, memory_format=torch.channels_last)
            with torch.amp.autocast('cuda', enabled=torch.cuda.is_available()):
                out = model(imgs)
            if out.ndim>2: out = out.flatten(1)
            out = torch.nn.functional.normalize(out.float(), dim=1)
            q = out.detach().cpu().numpy().astype(np.float32)
            D, I = index.search(q, 1)
            lbls = proto_lbls[I[:,0]]
            preds.append(lbls)
            if b % 50 == 0:
                print(f'Test batch {b}/{len(dl_te)} elapsed {time.time()-t1:.1f}s', flush=True)
    preds = np.concatenate(preds).astype(np.int32)
    print(f'Test queried in {time.time()-t1:.1f}s', flush=True)
    pred_cat = [lbl2cat[int(x)] for x in preds]
    sub = pd.DataFrame({ 'Id': pd.read_csv(SAMPLE_SUB)['Id'].values, 'Predicted': pred_cat })
    sub.to_csv('submission.csv', index=False)
    print('Retrieval submission.csv written:', sub.shape, flush=True)

print('Retrieval code ready (uses FAISS IndexFlatIP). To run after deps present: \n- pip install faiss-cpu (in a separate cell) \n- run_retrieval_submission(per_class=4)', flush=True)

Retrieval code ready (uses FAISS IndexFlatIP). To run after deps present: 
- pip install faiss-cpu (in a separate cell) 
- run_retrieval_submission(per_class=4)


In [18]:
# Install retrieval dependencies
import sys, subprocess, time, importlib
t0=time.time()
print('Installing faiss-cpu (and huggingface-hub) under torch constraints...', flush=True)
def pip(*args):
    print('>', *args, flush=True)
    subprocess.run([sys.executable, '-m', 'pip', *args], check=True)
pip('install', '-c', 'constraints.txt', 'faiss-cpu', 'huggingface-hub', '--upgrade-strategy', 'only-if-needed')
print('Installed in %.1fs' % (time.time()-t0), flush=True)
import faiss, timm
print('faiss version OK; timm version:', timm.__version__, flush=True)

Installing faiss-cpu (and huggingface-hub) under torch constraints...


> install -c constraints.txt faiss-cpu huggingface-hub --upgrade-strategy only-if-needed


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 183.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 563.3/563.3 KB 492.3 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 264.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 KB 350.7 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 KB 439.3 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 806.6/806.6 KB 358.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 KB 431.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 123.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.3/199.3 KB 469.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 KB 385.0 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.4/70.4 KB 348.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.3/150.3 KB 479.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.2/161.2 KB 497.5 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.8/129.8 KB 472.5 MB/s eta 0:00:00


Installed in 6.1s


faiss version OK; timm version: 1.0.9


In [20]:
# Run retrieval baseline to generate a fast submission
import time
t0=time.time()
print('Starting retrieval submission (per_class=4, test_batch=128)...', flush=True)
run_retrieval_submission(per_class=4, test_batch=128)
print('Retrieval done in %.1f min' % ((time.time()-t0)/60.0), flush=True)

Starting retrieval submission (per_class=4, test_batch=128)...


In [25]:
# Optimized sharded test inference (no TTA) per expert advice
import numpy as np, torch, time, pandas as pd, os
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from pathlib import Path

class TestDatasetSharded(Dataset):
    def __init__(self, df, tfms, start=0, end=None):
        self.df_full = df.reset_index(drop=True)
        self.start = int(start)
        self.end = int(end) if end is not None else len(self.df_full)
        self.df = self.df_full.iloc[self.start:self.end].reset_index(drop=True)
        self.tfms = tfms
    def __len__(self):
        return len(self.df)
    def __getitem__(self, i):
        j = i
        fp = self.df.iloc[j]['file_path']
        try:
            img = Image.open(fp).convert('RGB')
        except Exception:
            img = Image.fromarray(np.zeros((IMG_SIZE, IMG_SIZE, 3), dtype=np.uint8))
        img = self.tfms(img)
        return img, self.start + j

def _get_log_priors():
    # Compute log class priors from training data (align to NUM_CLASSES)
    counts = train_df['label'].value_counts().reindex(range(NUM_CLASSES), fill_value=0).values.astype(np.float32)
    counts = np.maximum(counts, 1.0)
    logp = np.log(counts)
    t = torch.from_numpy(logp).to(device)
    return t

def infer_fold_sharded(fold=0, batch_size=256, chunk_size=50000, out_memmap='test_predlbl.int32', tau=1.0):
    # tau: strength of prior correction; 0 disables
    ckpt_path = f'ckpt_fold{fold}.pt'
    print(f'Loading checkpoint: {ckpt_path}', flush=True)
    state = torch.load(ckpt_path, map_location='cpu')
    model = build_model()
    model.load_state_dict(state['model'], strict=True)
    ema = ModelEmaV2(model, decay=0.999, device=device)
    if 'ema' in state:
        ema.load_state_dict(state['ema'], strict=False)
    m = ema.module if 'ema' in state else model
    m.eval()
    test_df = build_test_df_in_submission_order()
    N = len(test_df)
    print('Test size:', N, flush=True)

    # Prepare/resume memmap
    mode = 'r+' if os.path.exists(out_memmap) else 'w+'
    mm = np.memmap(out_memmap, dtype='int32', mode=mode, shape=(N,))

    # Dataloader params per expert
    dl_kwargs = dict(batch_size=batch_size, shuffle=False, num_workers=16, pin_memory=True,
                     persistent_workers=False, prefetch_factor=4, drop_last=False)
    t_all = time.time()
    written = 0
    prior = _get_log_priors() if tau and tau > 0 else None
    with torch.inference_mode():
        for s in range(0, N, chunk_size):
            e = min(s + chunk_size, N)
            print(f'Chunk {s}:{e} ({e-s})', flush=True)
            ds = TestDatasetSharded(test_df, val_tfms, start=s, end=e)
            dl = DataLoader(ds, **dl_kwargs)
            t0 = time.time()
            for b, (imgs, idxs) in enumerate(dl):
                imgs = imgs.to(device, non_blocking=True, memory_format=torch.channels_last)
                with torch.amp.autocast('cuda', dtype=torch.float16, enabled=torch.cuda.is_available()):
                    logits = m(imgs)
                    if prior is not None:
                        logits = logits - float(tau) * prior
                pred = logits.argmax(1).detach().cpu().numpy().astype(np.int32)
                mm[idxs.numpy()] = pred
                written += len(pred)
                if b % 50 == 0:
                    print(f'  batch {b}/{len(dl)} wrote {written}/{N} elapsed {time.time()-t0:.1f}s', flush=True)
            mm.flush()
            Path('last_done.txt').write_text(str(e))
            print(f'Chunk {s}:{e} done in {time.time()-t0:.1f}s (total {time.time()-t_all:.1f}s)', flush=True)
    del mm
    # Map to category ids and write submission
    preds = np.memmap(out_memmap, dtype='int32', mode='r', shape=(N,))
    pred_cat = [lbl2cat[int(x)] for x in preds]
    sub = pd.DataFrame({'Id': pd.read_csv(SAMPLE_SUB)['Id'].values, 'Predicted': pred_cat})
    sub.to_csv('submission.csv', index=False)
    print('submission.csv written with shape', sub.shape, 'total time %.1fs' % (time.time()-t_all), flush=True)
    return 'submission.csv'

print('Sharded inference utility ready: call infer_fold_sharded(fold=0, batch_size=256, chunk_size=50000, tau=1.0) after training.', flush=True)

Sharded inference utility ready: call infer_fold_sharded(fold=0, batch_size=256, chunk_size=50000, tau=1.0) after training.


In [ ]:
# Launch sharded inference (no TTA) from best fold-0 checkpoint
import os, time, pandas as pd
t0=time.time()
print('Starting sharded inference: fold=0, bs=256, chunk=50k, tau=1.0', flush=True)
out = infer_fold_sharded(fold=0, batch_size=256, chunk_size=50000, tau=1.0)
assert os.path.exists('submission.csv'), 'submission.csv not found after inference'
sub = pd.read_csv('submission.csv')
print('submission.csv ready:', sub.shape, 'Elapsed: %.1fs' % (time.time()-t0), flush=True)
print(sub.head())

Starting sharded inference: fold=0, bs=256, chunk=50k, tau=1.0


Loading checkpoint: ckpt_fold0.pt


/tmp/ipykernel_222/793291016.py:38: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state = torch.load(ckpt_path, map_location='cpu')


Test size: 477806


Chunk 0:50000 (50000)
